<a href="https://colab.research.google.com/github/huuhung032/Re-Identification/blob/main/TestReid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

import os
os.chdir("/content/drive/MyDrive/deep-person-reid")

Mounted at /content/drive


In [ ]:
!pip install --upgrade quantecon

     |████████████████████████████████| 240 kB 5.3 MB/s 


In [ ]:
%load_ext Cython

In [ ]:
import torchreid
torchreid.models.show_avai_models()

['resnet18', 'resnet34', 'resnet50', 'resnet101', 'resnet152', 'resnext50_32x4d', 'resnext101_32x8d', 'resnet50_fc512', 'se_resnet50', 'se_resnet50_fc512', 'se_resnet101', 'se_resnext50_32x4d', 'se_resnext101_32x4d', 'densenet121', 'densenet169', 'densenet201', 'densenet161', 'densenet121_fc512', 'inceptionresnetv2', 'inceptionv4', 'xception', 'resnet50_ibn_a', 'resnet50_ibn_b', 'nasnsetmobile', 'mobilenetv2_x1_0', 'mobilenetv2_x1_4', 'shufflenet', 'squeezenet1_0', 'squeezenet1_0_fc512', 'squeezenet1_1', 'shufflenet_v2_x0_5', 'shufflenet_v2_x1_0', 'shufflenet_v2_x1_5', 'shufflenet_v2_x2_0', 'mudeep', 'resnet50mid', 'hacnn', 'pcb_p6', 'pcb_p4', 'mlfn', 'osnet_x1_0', 'osnet_x0_75', 'osnet_x0_5', 'osnet_x0_25', 'osnet_ibn_x1_0', 'osnet_ain_x1_0', 'osnet_ain_x0_75', 'osnet_ain_x0_5', 'osnet_ain_x0_25']


In [ ]:
# Construct data manager
datamanager = torchreid.data.ImageDataManager(
    root ='reid-data',
    sources = 'market1501',
    targets = 'market1501',
    height=256,
    width=128,
    batch_size_train=32,
    batch_size_test=100,
    transforms = ['random_flip', 'random_crop']
)

Building train transforms ...
+ resize to 256x128
+ random flip
+ random crop (enlarge to 288x144 and crop 256x128)
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
Building test transforms ...
+ resize to 256x128
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
=> Loading train (source) dataset
=> Loaded Market1501
  ----------------------------------------
  subset   | # ids | # images | # cameras
  ----------------------------------------
  train    |   751 |    12936 |         6
  query    |   750 |     3368 |         6
  gallery  |   751 |    15913 |         6
  ----------------------------------------
=> Loading test (target) dataset


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


=> Loaded Market1501
  ----------------------------------------
  subset   | # ids | # images | # cameras
  ----------------------------------------
  train    |   751 |    12936 |         6
  query    |   750 |     3368 |         6
  gallery  |   751 |    15913 |         6
  ----------------------------------------


  **************** Summary ****************
  source            : ['market1501']
  # source datasets : 1
  # source ids      : 751
  # source images   : 12936
  # source cameras  : 6
  target            : ['market1501']
  *****************************************




In [ ]:
# Construct OSnet model
model = torchreid.models.build_model(
    name='osnet_x1_0',
    num_classes=datamanager.num_train_pids,
    loss='softmax',
    pretrained=True
)

model = model.cuda()

# Initialise optimiser and learning rate scheduler
optimizer = torchreid.optim.build_optimizer(
    model,
    optim='adam',
    lr=0.0003
)

scheduler = torchreid.optim.build_lr_scheduler(
    optimizer,
    lr_scheduler='single_step',
    stepsize=20
)

Downloading...
From: https://drive.google.com/uc?id=1LaG1EJpHrxdAxKnSCJ_i0u-nbxSAeiFY
To: /root/.cache/torch/checkpoints/osnet_x1_0_imagenet.pth
100%|██████████| 10.9M/10.9M [00:00<00:00, 135MB/s]


Successfully loaded imagenet pretrained weights from "/root/.cache/torch/checkpoints/osnet_x1_0_imagenet.pth"
** The following layers are discarded due to unmatched keys or layer size: ['classifier.weight', 'classifier.bias']


In [ ]:
# Construct engine
engine = torchreid.engine.ImageSoftmaxEngine(
    datamanager,
    model,
    optimizer=optimizer,
    scheduler=scheduler,
    label_smooth=True,
)

In [ ]:
#Run model training and test
engine.run(
    save_dir='log/OSNet',
    max_epoch=20,
    eval_freq=10,
    print_freq=10,
    test_only=False,
)

=> Start training


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch: [1/20][10/404]	time 0.471 (1.840)	data 0.001 (1.316)	eta 4:07:27	loss 6.5664 (6.6552)	acc 0.0000 (0.0000)	lr 0.000300
epoch: [1/20][20/404]	time 0.478 (1.616)	data 0.001 (1.108)	eta 3:37:01	loss 6.5080 (6.6338)	acc 3.1250 (0.4688)	lr 0.000300
epoch: [1/20][30/404]	time 0.475 (1.613)	data 0.001 (1.117)	eta 3:36:28	loss 6.5407 (6.6125)	acc 3.1250 (1.2500)	lr 0.000300
epoch: [1/20][40/404]	time 0.480 (1.566)	data 0.000 (1.074)	eta 3:29:51	loss 6.4420 (6.5851)	acc 0.0000 (1.3281)	lr 0.000300
epoch: [1/20][50/404]	time 0.476 (1.500)	data 0.000 (1.010)	eta 3:20:45	loss 6.5408 (6.5596)	acc 0.0000 (1.8125)	lr 0.000300
epoch: [1/20][60/404]	time 0.475 (1.489)	data 0.001 (1.000)	eta 3:19:00	loss 6.5340 (6.5409)	acc 0.0000 (1.8229)	lr 0.000300
epoch: [1/20][70/404]	time 0.480 (1.482)	data 0.000 (0.994)	eta 3:17:49	loss 6.2538 (6.5158)	acc 3.1250 (2.0536)	lr 0.000300
epoch: [1/20][80/404]	time 0.479 (1.448)	data 0.001 (0.961)	eta 3:13:02	loss 6.2849 (6.4894)	acc 3.1250 (2.1484)	lr 0.000300
